### Imports

In [1]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool
from datetime import datetime
from bokeh.palettes import Spectral3
from tabulate import tabulate
from bokeh.transform import factor_cmap
import js2py
import math
from concurrent.futures import ProcessPoolExecutor as PoolExecutor
import requests
import os

# convenience
output_notebook()
pd.set_option('display.max_colwidth', -1)

Loading BokehJS ...

### Globals

In [2]:
API = os.getenv('API')
NUM_CHALLENGES = 100
NUM_BLOCKS = 100
TOP = 10
animalhash = js2py.require('angry-purple-tiger')
MAX_WORKERS = 8
REWARD_LIMIT = 500

### Helpers

In [3]:
def latest_height(api):
    '''
    Get latest block height
    '''
    r = requests.get(f'{api}/blocks?limit=1')
    if r.status_code == 200:
        data = r.json().get('data')
        return data[0]['height']
    raise Exception('Unable to get latest height')

def get_blocks(api, num_blocks=100):
    '''
    Get past `NUM_BLOCKS` number of num_blocks
    '''
    r = requests.get(f'{api}/blocks?limit={num_blocks}')
    if r.status_code == 200:
        data = r.json().get('data')
        assert len(data) == num_blocks
        return data
    raise Exception('Unable to get blocks')

def get_challenges(api, num_challenges=100):
    '''
    Get past `LIMIT` number of challenges
    '''
    r = requests.get(f'{api}/challenges?limit={num_challenges}')
    if r.status_code == 200:
        data = r.json().get('data')
        assert len(data) == num_challenges
        return data
    raise Exception('Unable to get challenges')

def get_hotspots(api):
    '''
    Get hotspots from the api
    '''
    r = requests.get(f'{api}/hotspots')
    if r.status_code == 200:
        data = r.json().get('data')
        return data
    raise Exception('Unable to get hotspots')

def get_accounts(api):
    '''
    Get accounts from the api
    '''
    r = requests.get(f'{api}/accounts')
    if r.status_code == 200:
        data = r.json().get('data')
        return data
    raise Exception('Unable to get accounts')

def accumulate_results(challenges):
    '''
    Return a dict list of challenge_id, num_successes, num_failures and num_untested
    '''
    results = []
    for challenge in challenges:
        path = challenge['pathElements']

        result = [element['result'] for element in path]
        length = len(result)
        num_untested = result.count('untested')

        gray = True if num_untested == length else False

        if gray:
            num_successes = 0
            num_failures = 0
            num_untested = 0
            dud = 1
            success = 0
        else:
            num_successes = result.count('success')
            num_failures = result.count('failure')
            num_untested = num_untested
            dud = 0
            success = 1 if num_successes == length else 0
        results.append({'id': challenge['id'],
                        'num_successes': num_successes,
                        'num_failures': num_failures,
                        'num_untested': num_untested,
                        'dud': dud,
                        'path_length': len(path),
                        'success': success})
    return results

def get_reward(reward_url):
    '''
    Get hotspot reward
    '''
    r = requests.get(reward_url)
    if r.status_code == 200:
        data = r.json().get('data')
        return data
    raise Exception(f'Unable to get rewards for {reward_url}')
    
def get_rewards(api, hotspots, limit=REWARD_LIMIT):
    '''
    Get rewards for all hotspots
    '''
    reward_urls = [f'{api}/hotspots/{hotspot}/rewards?limit={limit}' for hotspot in hotspots]
    with PoolExecutor(max_workers=MAX_WORKERS) as executor:
        rewards = []
        for hotspot, i in zip(hotspots, executor.map(get_reward, reward_urls)):
            rewards.extend(i)
    return rewards

### Block Stats

In [4]:
def block_stats():
    blocks = get_blocks(API, NUM_BLOCKS)

    df = pd.DataFrame(blocks)
    max_txns = tabulate(df.max().reset_index(),tablefmt='psql', showindex='never')
    print(f'Average txns past {NUM_BLOCKS} blocks: {df["txns"].mean()}')
    print(f'Average block time past {NUM_BLOCKS} blocks: {df["time"].mean()}')
    print(f"Block with max txns for past {NUM_BLOCKS} blocks: \n{max_txns}")
    return df
bdf = block_stats()

Average txns past 100 blocks: 4.93
Average block time past 100 blocks: 1565836320.63
Block with max txns for past 100 blocks: 
+--------+----------------------------------------------------+
| hash   | 1y6RdJLfETKcWMUeWBHvBbGfLbfiZDjjDhg2wHyctrfNAAL6eB |
| height | 16449                                              |
| round  | 16449                                              |
| time   | 1565839967                                         |
| txns   | 25                                                 |
+--------+----------------------------------------------------+


In [5]:
def plot_block_stats(df):
    source = ColumnDataSource(df)
    p = figure(title='Txns/Block')

    p.vbar(x='height', top='txns', width=0.8, source=source)
    hover = HoverTool(tooltips=[('height', '@height'), ('txns', '@txns')])
    p.add_tools(hover)
    p.xaxis.axis_label = "Txns/Block"
    p.yaxis.axis_label = "NumTxns"

    show(p)
plot_block_stats(bdf)

### Hotspot Stats

In [6]:
def hotspot_stats():
    hotspots = get_hotspots(API)

    hdf = pd.DataFrame(hotspots)
    hdf['name'] = hdf.apply(lambda row: '-'.join(animalhash(row.address).lower().split()), axis=1)
    print(f'Top {TOP} scoring hotspots:')
    print(tabulate(hdf.sort_values('score', ascending=False)[['name', 'score']].head(TOP),
                   headers='keys', tablefmt='psql'))
    return hdf
hdf = hotspot_stats()

Top 10 scoring hotspots:
+-----+-------------------------+---------+
|     | name                    |   score |
|-----+-------------------------+---------|
| 116 | magic-carob-quail       |  0.9939 |
| 133 | mean-alabaster-piranha  |  0.9466 |
| 103 | delightful-stone-beetle |  0.9329 |
| 132 | mythical-tin-hawk       |  0.919  |
|  13 | beautiful-onyx-platypus |  0.9154 |
| 130 | mini-currant-lizard     |  0.8971 |
| 126 | joyous-burgundy-chicken |  0.8956 |
|  60 | shambolic-rusty-cyborg  |  0.8937 |
|  44 | bubbly-mandarin-panther |  0.8811 |
|  43 | melodic-pearl-chameleon |  0.8217 |
+-----+-------------------------+---------+


In [7]:
def plot_hotspot_stats(df):
    source = ColumnDataSource(df)
    p = figure(x_range=df['name'], y_range=[0,1], title="Hotspot Scores")

    p.vbar(x='name', top='score', width=0.8, source=source)
    hover = HoverTool(tooltips=[('name', '@name'), ('score', '@score')])
    p.width = 900
    p.add_tools(hover)
    p.xaxis.axis_label = "Name"
    p.yaxis.axis_label = "Score"
    p.xaxis.major_label_text_font_size = "4pt"
    p.xaxis.major_label_orientation = math.pi/4

    show(p)
plot_hotspot_stats(hdf)

### Account stats

In [8]:
def account_stats():
    accounts = get_accounts(API)
    df = pd.DataFrame(accounts)
    df['balance'] = df.apply(lambda row: row.balance/100000000, axis=1)
    print(f'Top {TOP} account balances:')
    print(tabulate(df.sort_values('balance', ascending=False)[['address', 'balance']].head(TOP),
                   headers='keys', tablefmt='psql'))
    return df
adf = account_stats()

Top 10 account balances:
+----+-----------------------------------------------------+-----------+
|    | address                                             |   balance |
|----+-----------------------------------------------------+-----------|
| 42 | 13jL4pTLq4JQBwDoH1ZBX3VPchmeTZxQSGi4cnyXXuzPPoKxcd5 |   64311.2 |
| 69 | 1398hLeHESZHE5jVtaLAV5fdg2vrUeZEs2B92t7TzeQTtugr8dL |   45156.7 |
| 97 | 14a5sytsEVKBVhGsVz2rJDkxf1cs1L89zvtFjmPtmPzrQHXPVjF |   37926   |
| 82 | 14GWyFj9FjLHzoN3aX7Tq7PL6fEg4dfWPY8CrK8b9S5ZrcKDz6S |   21374.3 |
| 85 | 12yyntxFfPhqXDKuEvmAWABNTRnXtYmcgS33WN4tk5YtyaBLDXo |   21347.8 |
| 49 | 13o9KUMcmEdhtjXRaJ5AANHLZWBoGcTmF4MFw3MjQbubjR57jXW |   19091.6 |
| 81 | 12z4nUiayZnbW46azcegBBB9yGkchFb1Zm7EPuh6eV1r2HT1NgC |   12842.8 |
| 75 | 14i6nM1RbWbjQFg5NFbZS3aZJgpUGZUx5toarz6jZLPpNAb95Tq |   12548.3 |
| 89 | 14UgUhAPq53iR8vfde6sU4vLPWRe6kZ3ioV7xaVb8hr3oafaok6 |   12213.9 |
| 78 | 135bB6B7gGrsdPNcFwvcn12KPjTktSNZvmVbvSBKubCtEgxkpPJ |   11861.3 |
+----+--------------------

### Challenge Stats

In [9]:
def challenge_stats():
    challenges = get_challenges(API, NUM_CHALLENGES)
    res = accumulate_results(challenges)
    df = pd.DataFrame(res)

    print(f"Total Successful challenges past {NUM_CHALLENGES} challenges: {sum(df['success'] == 1)}")
    print(f"Entirely dud challenges since past {NUM_CHALLENGES} challenges: {sum(df['dud'] == 1)}")
    return df
cdf = challenge_stats()

Total Successful challenges past 100 challenges: 10
Entirely dud challenges since past 100 challenges: 27


In [10]:
def plot_challenge_stats(df):
    source = ColumnDataSource(df)
    p = figure(title='Challenge Paths')

    p.vbar(x='id', top='path_length', width=0.8, source=source)
    hover = HoverTool(tooltips=[('id', '@id'), ('path_length', '@path_length')])
    p.add_tools(hover)
    p.yaxis.axis_label = "PathLength"
    p.xaxis.axis_label = "ChallengeId"

    show(p)
plot_challenge_stats(cdf)

### Hotspot Reward Stats

In [11]:
def hotspot_reward_stats():
    rewards = get_rewards(API, hdf['address'].values)
    rdf = pd.DataFrame(rewards)[['gateway', 'amount', 'account']]
    g1 = rdf.groupby(['gateway', 'account']).sum().reset_index()
    g1['name'] = g1.apply(lambda row: '-'.join(animalhash(row.gateway).lower().split()), axis=1)
    g1['amount'] = g1.apply(lambda row: row.amount/100000000, axis=1)
    print(f'Top {TOP} rewarded hotspots:')
    print(tabulate(g1.sort_values('amount', ascending=False)[['name', 'amount', 'account']].head(TOP),
                   headers='keys', tablefmt='psql'))
    return rdf, g1
rdf, g = hotspot_reward_stats()

Top 10 rewarded hotspots:
+-----+-------------------------+----------+-----------------------------------------------------+
|     | name                    |   amount | account                                             |
|-----+-------------------------+----------+-----------------------------------------------------|
| 109 | shambolic-rusty-cyborg  | 18622.9  | 13jL4pTLq4JQBwDoH1ZBX3VPchmeTZxQSGi4cnyXXuzPPoKxcd5 |
|  55 | mean-alabaster-piranha  | 12014.8  | 14a5sytsEVKBVhGsVz2rJDkxf1cs1L89zvtFjmPtmPzrQHXPVjF |
|   8 | mythical-tin-hawk       | 11216    | 14a5sytsEVKBVhGsVz2rJDkxf1cs1L89zvtFjmPtmPzrQHXPVjF |
|  94 | clean-nylon-okapi       | 10950.1  | 13jL4pTLq4JQBwDoH1ZBX3VPchmeTZxQSGi4cnyXXuzPPoKxcd5 |
|  26 | fit-iron-monkey         |  8258.26 | 14a5sytsEVKBVhGsVz2rJDkxf1cs1L89zvtFjmPtmPzrQHXPVjF |
|  47 | cuddly-sapphire-narwhal |  7758.87 | 14i6nM1RbWbjQFg5NFbZS3aZJgpUGZUx5toarz6jZLPpNAb95Tq |
| 114 | teeny-cider-troll       |  7491.28 | 13o9KUMcmEdhtjXRaJ5AANHLZWBoGcTmF4MFw3

In [12]:
def plot_challenge_stats(g1):
    source = ColumnDataSource(g1)
    p = figure(x_range=g1['name'], title=f"Aggregated last {REWARD_LIMIT} hotspot rewards")

    p.vbar(x='name', top='amount', width=0.8, source=source)
    hover = HoverTool(tooltips=[('name', '@name'), ('amount', '@amount')])
    p.width = 900
    p.add_tools(hover)
    p.xaxis.axis_label = "Hotspot"
    p.yaxis.axis_label = "AggregateRewards"
    p.xaxis.major_label_text_font_size = "4pt"
    p.xaxis.major_label_orientation = math.pi/4

    show(p)
plot_challenge_stats(g)